# Custom Export

In this example we take metadata and data from BMM and export it in the [XDI](https://github.com/XraySpectroscopy/XAS-Data-Interchange/blob/master/specification/spec.md) format.

In [ ]:
from tiled.client import from_uri

In [ ]:
c = from_uri("https://tiled-demo.blueskyproject.io/api")
c

In [ ]:
run = c['bmm']['raw'][22521]
run

In [ ]:
df = run['primary']['data'].read().to_dataframe()
df

In [ ]:
xdi_metadata = run.metadata['start']['XDI']
xdi_metadata

In [ ]:
def write_xdi(output, df, metadata):

    xdi_version = metadata.pop("xdi_version", "1.0")
    comments = metadata.pop("comments", "")
    extra_version = metadata.pop("extra_version", "")

    output.write(f"# XDI/{xdi_version} {extra_version}\n")

    for namespace, namespace_dict in metadata.items():
        if namespace.startswith("_"):
            continue
        for tag, value in namespace_dict.items():
            output.write(f"# {namespace}.{tag}: {value}\n")

    output.write("# /////////////\n")
    output.write("# generated by tiled\n")
    output.write(comments)
    output.write("# -------------\n")

    # write column labels
    columns = list(df.columns)
    output.write("# ")
    output.write(" ".join(columns))
    output.write("\n")

    # write data
    df.to_csv(output, header=False, index=False)

In [ ]:
with open("my_data.xdi", "w") as file:
    write_xdi(file, df, xdi_metadata)